# 映画またはテレビ番組が賞にノミネートされる、または受賞するかの予測

2005 年、リードデータサイエンティストとして次のような課題に直面しています。Amazon Studios は映画賞を受賞するような作品を複数制作したいと考えています。受賞の可能性を最大限に引き出すために、予算をプロジェクトに集中しさせています。Amazon の子会社である IMDb の実際のデータセットを使用して、1990 年から 2005 年までに制作された映画について調査します。

IMDb データセットは、この期間中に公開されたすべての映画に関する詳細で包括的なリストです。データセットには、映画の出演者、裏方、作品の概要、その他の制作データなど、重要なデータが含まれています。 このデータの多くはパブリックな IMDb.com サイトで公開され、公開されていないものはスタジオ分析のために非公開となっています。

賞分析予測モデルを構築して、来るべき 2005年の賞シーズン中にノミネートされる可能性が最も高い映画を予測します。

このノートブックにより、IMDB データベースからのデータのフィールドを読み取り、映画が賞に "ノミネートされる" または "受賞する" かを予測するためのモデルを構築します。

このデータセットは IMDb の許可により提供されており、[AWS デジタルトレーニングサービス契約](https://aws.amazon.com/training/digital-training-agreement) の条項に従う必要があります。 このラボの実施以外の目的でデータセットをコピー、変更、販売、エクスポート、使用することは明示的に禁止されています。

## 必要なライブラリをインポートする

ここでは、ライブラリを使ってモデルを事前処理し、予測を立てます
（必要なほかのライブラリを追加することもできます）。

In [ ]:
# Importing libs into the python environment. These functions will be referenced later in the notebook code.

from __future__ import print_function
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import gzip
import numpy as np
import seaborn as sns
import itertools
from IPython.display import Markdown, display
from mpl_toolkits.mplot3d import axes3d, Axes3D  # <-- Note the capitalization!
%matplotlib inline

sns.set()


# Modules from scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report

from sklearn import decomposition
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.preprocessing import StandardScaler


## データについての考慮事項

IMDB データベースには大量の情報が含まれています。そのため、予測のどの種類のデータを用いるかが重要となります。使用するスキームは以下のようになります


<img src="Data-Schema-Capstone.png">

## データをクリーニングおよび可視化する

**<LabBucketName\>** をラボアカウントで提供されているリソース名と入れ変えます。

In [ ]:
import boto3
import botocore 
bucket = '<LabBucketName>' # Update this to the bucket that was created in your lab account as part of this enviroment.
prefix = 'data/'
 
s3 = boto3.resource('s3') 


Raw データファイルは AWS ラボアカウントの S3 バケットにあります。6 つのテーブルが使用されます（`title_genres`、`title_ratings`、`title_display`、`award_noms`、`title_awards`、`title_releases`）。Raw タブ区切り値ファイルが Amazon Sagemaker インスタンスにダウンロードされ、構造化データとの連動が簡単な DataFrame にインポートされます。Raw ファイルは行ヘッダーには含まれず、ラベルはインポート時に割り当てられています。

In [ ]:
def download_and_display_file(filename,names, title):
    s3.Bucket(bucket).download_file(filename, filename)
    user_info = pd.read_csv(filename, sep='\t', encoding= 'latin1', names = names)
    display(Markdown("**" + title +" Table** \n"))
    display(user_info.head(5))
    return user_info

user_info_genres = download_and_display_file('title_genres.tsv', ['titleId','genres'], 'Genres')
user_info_ratings = download_and_display_file('title_ratings.tsv', ["titleId","rating","ratingCount","topRank","bottomRank","topRankTV"], 'Rating')
user_info_display = download_and_display_file('title_display.tsv', ["titleId","title","year","adult","runtimeMinutes","imageUri","imageId","type","originalTitle"], 'Display')
user_info_noms = download_and_display_file('award_noms.tsv', ["awardId","eventId","event","eventEditionId","award","category","year"], 'Nomination')
user_info_awards = download_and_display_file('title_awards.tsv', ["titleId","awardId","winner"], 'Awards')
user_info_releases = download_and_display_file('title_releases.tsv', ["titleId","ordering","date","region","premiere","wide","premiereType","festival","attributes"], 'Releases')



テーブル形式 (.tsv) のデータはデータの事前処理のため Pandas DataFrame 内で使用されます。データは、6 つの異なるファイル間で、または DataFrames に分割されます。データをマージして、さらなるデータ探査、データエンジニアリング、可視化、およびモデル構築で使用される 統合された DataFrame 取得します。pandas の組み込み `merge` 関数を使って、DataFrames を一緒にマージします。`TitleId` は一意の ID で、このデータセットの各映画のタイトルに割り当てられています。`ttle_ratings`、`title_genres`、`title_display`、`title_releases` 間の一連の内部結合はすべてのテーブルを共にマージします。

In [ ]:
df_first_merge = pd.merge(user_info_genres, user_info_ratings, on='titleId', how='inner')
df_second_merge = pd.merge(df_first_merge, user_info_display, on='titleId', how='inner')
df_third_merge = pd.merge(df_second_merge, user_info_releases, on='titleId', how='inner')

(title_awards) テーブルとの外部結合を行う前に、重複した titleId が削除されます。DataFrame の結果として生じた重複は、いくつかのデータ列のとともに削除されます。IMDB データセットは数百ものフィールドが巨大なものです。ここでは、モデルの出力に影響を与える可能性がある関連するフィールドのみが選択されています。これらの関連するテーブルを読みこんだ後、次に示すように関連するフィールドのみが保持され、残りのフィールドは削除されます。

In [ ]:
df_third_merge = df_third_merge.drop_duplicates(['titleId'])
df_fourth_merge = pd.merge(df_third_merge,user_info_awards,on='titleId', how='outer' )
    
df = df_fourth_merge.drop_duplicates(['titleId'])
df = df.drop(['imageUri','topRank','bottomRank','topRankTV','ordering','premiereType','festival' ], axis=1)

結果 DataFrame はシリアル化され、**Pickle** ライブラリを使用して Pickle と呼ばれるフラットファイルに書き込まれます。その後、このファイルはデータをあとで再使用するために Amazon S3 バケットに保存されます。`pickle.dump` を使って Pickle ファイルを作成し、raw データファイルをオブジェクトに保存して、ラボの S3 バケットにファイルをアップロードできます。

In [ ]:
with open('df_pickle_nonoms_new.pkl', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)
s3.Bucket(bucket).upload_file('df_pickle_nonoms_new.pkl','data/df_pickle_nonoms_new.pkl')

最適化されたテーブルの上位 30 行を確認します。

In [ ]:
df.head(30)

新しい DataFrame でいくつかの基本的な pandas の記述統計を実行します。 

`df.info()` は DataFrame に関する詳細情報を出力します。これには、インデックス dtype、列 dtypes、null 以外の値、メモリ使用量などの情報が含まれます。`df.describe()` は、平均値、中央値、最頻値などの記述統計を生成します。

In [ ]:
df.info()

In [ ]:
df.describe()

Pickle ファイルを Pandas Dataframe に読み込み、無関係な作品の一部を削除します。

In [ ]:
s3.Bucket(bucket).download_file('data/df_pickle_nonoms_new.pkl', 'df_pickle_nonoms_new.pkl')
df = pickle.load(open('df_pickle_nonoms_new.pkl', 'rb'))
df = df[df.type == 'movie']
df = df.drop(['imageId', 'originalTitle', 'awardId', 'attributes' ], axis=1)

結果データは正規化が必要です。映画の上映時間がわからない場合は、ソースデータの値として `\N` があります。データをプロットしようとする場合は、この null 値では問題が発生します。runtime の '\N' を 0 に変更します。 

同様に、映画のリリース年が不明な場合、値は '\N' に設定されます。下のセルで、year の '\N' を 0 に変更します。

以下に runtimeMinutes '\N' のテーブルを示します。

In [ ]:
df[df.runtimeMinutes == r'\N'].head()

以下に year `\N` のテーブルを示します。

In [ ]:
df[df.year == r'\N'].head()

In [ ]:
for i, mins in df['runtimeMinutes'].iteritems():
    if mins == r'\N':
        better_name = '0'
        df.loc[[i],['runtimeMinutes']] = better_name

for i, year in df['year'].iteritems():
    if year == r'\N':
        better_name = '0'
        df.loc[[i],['year']] = better_name


 DataFrame に `nomination_winner` という別の列が追加されます。winner の列が `0.0` または `1.0` の場合、その作品がノミネートされていると推測されます。それ以外の場合、作品はノミネートされていません。

In [ ]:
df['nomination_winner'] = 0
for i, winner in df['winner'].iteritems():
    if winner == (0.0):
        better_name = 1
        df.loc[[i],['nomination_winner']] = better_name
    if winner == (1.0):
        better_name = 1
        df.loc[[i],['nomination_winner']] = better_name

また `fillna` 関数を使用して、`year` 列と `runtimeMinutes` 列の欠損値を入力します。

In [ ]:
df.runtimeMinutes = df.runtimeMinutes.astype(float).fillna(0.0)
df.year = df.year.astype(int).fillna(0.0)

データセットに含まれる一部の作品には、上映時間が一致しないと思われるものがあります。runtime が 1 時間（60 分）より長く、12 時間（720 分）以下のデータに制限します。また `reviewCount` が 20,000 未満の作品にも注力します。

以下に `runtimesMinutes` が 60 分未満のサンプルデータを示します。

In [ ]:
df[(df.runtimeMinutes) < 60].head()

以下に `runtimesMinutes` が 720 分未満のサンプルデータを示します。

In [ ]:
df[(df.runtimeMinutes) > 720].head()

In [ ]:
df = df[(df.runtimeMinutes) > 60]
df = df[(df.runtimeMinutes) < 720]
df = df[(df.ratingCount) < 20000]

**最適化されたデータの上位 30 列を確認します。**

In [ ]:
df.head(30)

In [ ]:
print('='*50)
df.info()
print('='*50)
df.describe()

**以下で、将来使用するために 2005 年のデータフレームを df_2005 として保存しています。**

In [ ]:
df_2005 = df[(df.year) == 2005]
df_2005.head()

## 特徴量選択と特徴量エンジニアリング
<a id='feature selection'></a>

**特徴量選択ボックス:** さまざまな特徴量と選択の切り替えスイッチは以下の通りです。値 0 は特徴量を無効化し、1 は有効化します。

In [1]:
# Selection of different features

feature_winner = 0          # Select this feature to make prediction on award winner. 
                            # Disable this feautre to make prediction on nomination winners.
    
feature_pca_2D = 0          # Select this feature to perform Principal Component Analysis of 2 components.
feature_pca_3D = 1          # Select this feature to perform Principal Component Analysis of 3 components.

feature_premiere = 0        # Select this feature to limit analysis on limited premiered movies.
feature_wide = 0            # Select this feature to limit analysis on world wide premiered movies.
feature_premiere_wide = 1   # Select this feature to include analysis on both limited and wide premiered movies.


# Normalize features
normalize_flag = 0

#Enable plotting
plot_flag = 1                 

#Feaure Selection 
US_flag = 1                 # Select this feature to limit analysis on US based movies.

#Model Selection flags
LR_flag = 1
DT_flag = 1
RF_flag = 1
GB_flag = 1
NN_flag = 1
SVM_flag = 1


**データのクリーンアップと制約:** 
- データを **US** 特徴のに見制限して探索します。
- 出力ターゲットとして **ノミネート** を使用するか、**受賞**  を使用するかを選択して探索します。

In [ ]:
if plot_flag: 
    prob = df.region.value_counts(normalize=True)
    threshold = 0.02
    mask = prob > threshold
    tail_prob = prob.loc[~mask].sum()
    prob = prob.loc[mask]
    prob['other'] = tail_prob
    prob.plot(kind='bar')
    plt.xticks(rotation=25)
    plt.show()
    plt.close()



if US_flag:
    df = df[ (df.region) == 'US']


# This flag is set if "winner" is chosen as output label

if feature_winner:
    df = df[(df.nomination_winner) == 1]

    for i, winner in df['winner'].iteritems():
        if winner == (0.0):
            better_name = 0
            df.loc[[i],['nomination_winner']] = better_name
        if winner == (1.0):
            better_name = 1


この特徴をグラフ化して、特徴が相互にどのように関連しているかわかりやすくします。以下は、データがインポートされた後に呼び出される関数を定義しています。

In [ ]:
#plot Confusion Matrix function

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()
    plt.close()


# end Confusion Matrix function


# ROC Curve Plotting function

def plot_roc_curve(fpr,tpr):
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b',
             label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([-0.1, 1.2])
    plt.ylim([-0.1, 1.2])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    plt.close()

#end ROC plotting function


# Precision Recall (PR) function

def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])
    plt.show( )
    plt.close()

#end PR function
 

以下では、`rating`、`ratingCount` などの異なる特徴のヒストグラムをプロットします。出力の `premier` フラグの効果を確認します。また出力だけでなく、さまざまな特徴間の影響や相互関係も調べます。`ratingCount` は、出力 `nomination_winner` (`nomination` または `winner`) に最大の影響することがわかります。

In [ ]:

# Plot rating vs rating count & Histogram of Rating

if plot_flag:
    
    df['rating'].hist()
    plt.title('Histogram of Ratings')
    plt.xlabel('Rating')
    plt.ylabel('Count')
    plt.show()
    plt.close()
    
    
    df_plot = df[(df.ratingCount) < 10000]
    df_plot['ratingCount'].hist(bins=100)
    plt.title('Histogram of Rating Count')
    plt.xlabel('Rating Count')
    plt.ylabel('Count')
    plt.show()
    plt.close()

    plot_hist1 = df[(df.nomination_winner) == 1]
    plot_hist0 = df[(df.nomination_winner) == 0]
    plt.hist(df.nomination_winner)
    plt.hist(plot_hist1.premiere)
    plt.xlabel('Nomination/Premiere')
    plt.ylabel('Count')
    plt.show()
    plt.close()

 
    #Plotting correlation  

    
    sns.pairplot(df[['nomination_winner', 'ratingCount','rating', 'runtimeMinutes' ]].head(5000));
    plt.show()
    plt.close()


**** 異なる特徴を試してみて、各モデルの roc スコアを確認します**。 ** 特徴量選択ボックス**の手順を実行します。

<a href='#feature selection'>**特徴量選択ボックスへ移動**</a>

In [ ]:

if feature_premiere_wide:
    X_train = df[['rating', 'ratingCount', 'runtimeMinutes','premiere','wide']]
elif feature_premiere:
    X_train = df[['rating', 'ratingCount', 'runtimeMinutes','premiere' ]]
elif feature_wide:
    X_train = df[['rating', 'ratingCount',  'runtimeMinutes' ,'wide']]
else :
    X_train = df[['rating', 'ratingCount',  'runtimeMinutes']]


Y_train = df['nomination_winner']

if normalize_flag:
    X_train=(StandardScaler().fit_transform(X_train ))


以下で、**主成分分析 (PCA)** を実行して、特徴セットを 3 つの主成分に減らします。

In [ ]:
#PCA


if feature_pca_3D:
    pca = decomposition.PCA(n_components=3)
    
     
    principalComponents = pca.fit_transform(X_train )

    principalDf = pd.DataFrame(data = principalComponents
                 , columns = ['principal component 1', 'principal component 2','principal component 3'])
    
    
    finalDf = pd.concat([principalDf, Y_train], axis = 1)
    finalDf = finalDf.head(2000)
    targets = [1, 0 ]
    colors = ['r', 'g' ]
     
    my_dpi = 96
    
    fig = plt.figure(figsize=(10,6))
    
    ax = fig.add_subplot(111,projection='3d' )
    ax.set_xlabel('Principal Component 1', fontsize = 8)
    ax.set_ylabel('Principal Component 2', fontsize = 8)
    ax.set_zlabel('Principal Component 3', fontsize = 8)
    ax.set_title('3 component PCA', fontsize = 15)
    for target, color in zip(targets,colors):
        indicesToKeep = finalDf['nomination_winner'] == target
        
        
        ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
                    , finalDf.loc[indicesToKeep, 'principal component 2']
                   ,  finalDf.loc[indicesToKeep, 'principal component 3']
                    , c = color,  linewidth=0.5)
        
    ax.legend(targets)
    #ax.grid() 
    plt.show()
    plt.close()
    
    X_train_pca = pca.transform(X_train)
    
    # In order to use PCA_3D for testing, uncomment below line and run this cell.
    #X_train = X_train_pca 
    
elif feature_pca_2D:
    
     
    pca = decomposition.PCA(n_components=2)
    principalComponents = pca.fit_transform(X_train )

    
    
    principalDf = pd.DataFrame(data = principalComponents
                 , columns = ['principal component 1', 'principal component 2'] )
    
    finalDf = pd.concat([principalDf, Y_train], axis = 1)
    finalDf = finalDf.head(50)
    
    targets = [1, 0 ]
    colors = ['r', 'g' ]
     
    my_dpi = 96
  
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111 )

    ax.set_xlabel('Principal Component 1', fontsize = 8)
    ax.set_ylabel('Principal Component 2', fontsize = 8)

    ax.set_title('2 component PCA', fontsize = 15)
    for target, color in zip(targets,colors):
        indicesToKeep = finalDf['nomination_winner'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
                   , finalDf.loc[indicesToKeep, 'principal component 2']               
                    , c = color,  linewidth=0.5)

    ax.legend(targets)
    #ax.grid() 
    plt.show()
    plt.close()
    
    X_train_pca = pca.transform(X_train)
    
    # In order to use PCA_2D for testing, uncomment below line and run this cell.
    #X_train = X_train_pca 
    
  

#PCA ends

"ノミネート/受賞" 3D ポイント (赤いドット) が、非"ノミネート/受賞" 3D ポイント (緑のドット) のクラスター内にあることが、分析中に示されます。これは、PCA 分析を使用して 3D モデルのデータを分類するのが難しいことを表しています。

データセットをトレーニングデータとテストデータのセットに分割します。

In [ ]:

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)
 

## アルゴリズムの比較と選択

Logistic Regression、Support Vector Machine (SVM)、Random Forest (RF)、Decision Tree (DT)、Gradient Boosting (GB)、Multi-Layer Perceptron (NN) 分類スキームを使用してモデルを構築します。Precision、Recall、ROC、F1、Accuracy などのさまざまなスコアが測定されます。また、異なるモデルの ROC 曲線をプロットします。

In [ ]:
if LR_flag:

# Logistic Regression Model

    logisticRegr = LogisticRegression(random_state=0, solver='lbfgs',
                        multi_class='multinomial', max_iter=1000)
    logisticRegr.fit(x_train, y_train)
    y_test_pred_LR = cross_val_predict(logisticRegr, x_test, y_test, cv=3)
    score = logisticRegr.score(x_test, y_test)
    y_test_pred = y_test_pred_LR
    print("LR Accuracy",accuracy_score(y_test, y_test_pred))
    roc = roc_auc_score(y_test, y_test_pred)
    print("roc score", roc)
    print("Classification Report")
    print("="*50)
    LR_CR = classification_report(y_test, y_test_pred)
    print(classification_report(y_test, y_test_pred))
    # Plot   confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    np.set_printoptions(precision=2)

    plot_confusion_matrix(cnf_matrix, classes=['Class 0', 'Class 1'],
                      title='Confusion matrix, without normalization')
    
    # Compute ROC curve and ROC area
    #fpr, tpr, thrsehold = roc_curve(y_test , y_test_pred)
  

    
    logit_roc_auc = roc_auc_score(y_test_pred, logisticRegr.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, logisticRegr.predict_proba(x_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression ' )
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
     
    plt.show()


In [ ]:

if SVM_flag:
    # SVM Model

    sgd_clf = SGDClassifier(random_state=0, loss="log", max_iter=1000, tol=3 )
    sgd_clf.fit(x_train, y_train)
    cross_val_score(sgd_clf, x_train, y_train, cv=3, scoring="accuracy")
    y_test_pred_SVC = cross_val_predict(sgd_clf, x_test, y_test, cv=3)
    predictions = sgd_clf.predict(x_test)
    score = sgd_clf.score(x_test, y_test)
    y_test_pred = y_test_pred_SVC
    print("SVM Accuracy",accuracy_score(y_test, y_test_pred))
    roc = roc_auc_score(y_test, y_test_pred_SVC)
    print("roc score", roc)
    SVM_CR = classification_report(y_test, y_test_pred)
    print("Classification Report")
    print("="*50)
    print(SVM_CR)
    # Plot   confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    np.set_printoptions(precision=2)

    plot_confusion_matrix(cnf_matrix, classes=['Class 0', 'Class 1'],
                      title='Confusion matrix, without normalization')
    
    # Compute micro-average ROC curve and ROC area
    fpr, tpr, thrsehold = roc_curve(y_test , y_test_pred)
     
  
    
    
    roc_auc = roc_auc_score(y_test_pred, sgd_clf.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, sgd_clf.predict_proba(x_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='SVM  ' )
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
     
    plt.show()



In [ ]:


if DT_flag:
    # Decision Tree Model

    DT = DecisionTreeClassifier(random_state=42)
    DT.fit(x_train, y_train)
    cross_val_score(DT, x_train, y_train, cv=3, scoring="accuracy")
    y_test_pred_DT = cross_val_predict(DT, x_test, y_test, cv=3)
    predictions = DT.predict(x_test)
    score = DT.score(x_test, y_test)
    y_test_pred = y_test_pred_DT
    print("DT Accuracy",accuracy_score(y_test, y_test_pred))
    roc = roc_auc_score(y_test, y_test_pred)
    print("roc score", roc)
    DT_CR = classification_report(y_test, y_test_pred)
    print("Classification Report")
    print("="*50)
    print(DT_CR)
    # Plot   confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    np.set_printoptions(precision=2)

    plot_confusion_matrix(cnf_matrix, classes=['Class 0', 'Class 1'],
                      title='Confusion matrix, without normalization')
    
    # Compute micro-average ROC curve and ROC area
    fpr, tpr, thrsehold = roc_curve(y_test , y_test_pred)
 
    
 
    
    roc_auc = roc_auc_score(y_test_pred, DT.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, DT.predict_proba(x_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Decision Tree '  )
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
     
    plt.show()   


In [ ]:



if RF_flag:
    # Ensemble Random Forest Model

    RF = RandomForestClassifier(random_state=42, n_estimators=100)
    RF.fit(x_train, y_train)
    cross_val_score(RF, x_train, y_train, cv=3, scoring="accuracy")
    y_test_pred_RF = cross_val_predict(RF, x_test, y_test, cv=3)
    predictions = RF.predict(x_test)
    score = RF.score(x_test, y_test)
    y_test_pred = y_test_pred_RF
    print("RF Accuracy",accuracy_score(y_test, y_test_pred))
    roc = roc_auc_score(y_test, y_test_pred)
    print("roc score", roc)
    RF_CR = classification_report(y_test, y_test_pred)
    print("Classification Report")
    print("="*50)
    print(RF_CR)
    # Plot   confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    np.set_printoptions(precision=2)

    plot_confusion_matrix(cnf_matrix, classes=['Class 0', 'Class 1'],
                      title='Confusion matrix, without normalization')
    
    
    # Compute micro-average ROC curve and ROC area
    fpr, tpr, thrsehold = roc_curve(y_test , y_test_pred)
 
    roc_auc = roc_auc_score(y_test_pred, sgd_clf.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, sgd_clf.predict_proba(x_test)[:,1])
 
    
    
    roc_auc = roc_auc_score(y_test_pred, RF.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, RF.predict_proba(x_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Random Forest ' )
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
     
    plt.show()




In [ ]:

if GB_flag:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=0.5, max_features=2, max_depth=2, random_state=0)
    gb_clf.fit(x_train, y_train)

    cross_val_score(gb_clf, x_train, y_train, cv=3, scoring="accuracy")
    y_test_pred_gb = cross_val_predict(gb_clf, x_test, y_test, cv=3)
    predictions = gb_clf.predict(x_test)
    score = gb_clf.score(x_test, y_test)
    y_test_pred = y_test_pred_gb
    print("GB Accuracy",accuracy_score(y_test, y_test_pred))
    roc = roc_auc_score(y_test, y_test_pred)
    print("roc score", roc)
    GB_CR = classification_report(y_test, y_test_pred)
    print("Classification Report")
    print("="*50)
    print(GB_CR)
    
    # Plot   confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    np.set_printoptions(precision=2)

    plot_confusion_matrix(cnf_matrix, classes=['Class 0', 'Class 1'],
                      title='Confusion matrix, without normalization')
    
    
    # Compute micro-average ROC curve and ROC area
    fpr, tpr, thrsehold = roc_curve(y_test , y_test_pred)
    roc_auc  = auc(fpr , tpr )
 
 
    roc_auc = roc_auc_score(y_test_pred, gb_clf.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, gb_clf.predict_proba(x_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='Gradient Boosting  '  )
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
     
    plt.show()


In [ ]:
if NN_flag:


    mlp = MLPClassifier(hidden_layer_sizes=(20, 20, 20), max_iter=1000)
    mlp.fit(x_test, y_test)

    cross_val_score(mlp, x_train, y_train, cv=3, scoring="accuracy")

    y_test_pred_NN = cross_val_predict(mlp, x_test, y_test, cv=3)
    predictions = mlp.predict(x_test)
    score = mlp.score(x_test, y_test)

    y_test_pred = y_test_pred_NN
    print("NN Accuracy",accuracy_score(y_test, y_test_pred))
    roc = roc_auc_score(y_test, y_test_pred)
    print("roc score", roc)
    NN_CR = classification_report(y_test, y_test_pred)
    print("Classification Report")
    print("="*50)
    print(NN_CR)
    
    # Plot   confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    np.set_printoptions(precision=2)

    plot_confusion_matrix(cnf_matrix, classes=['Class 0', 'Class 1'],
                          title='Confusion matrix, without normalization')

    # Compute micro-average ROC curve and ROC area
    fpr, tpr, thrsehold = roc_curve(y_test , y_test_pred)
    roc_auc  = auc(fpr , tpr )
  
 
    
     
    roc_auc = roc_auc_score(y_test_pred, mlp.predict(x_test))
    fpr, tpr, thresholds = roc_curve(y_test, mlp.predict_proba(x_test)[:,1])
    plt.figure()
    plt.plot(fpr, tpr, label='MLP  '  )
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig('Log_ROC')
    plt.show()

Below will display classification report of respective models.

In [ ]:
report_list = [LR_CR,SVM_CR,DT_CR,RF_CR,GB_CR,NN_CR]
model_list = ['LR','SVM','DT','RF','GB','NN']
def display_report(model,report):
    display(Markdown(model+" **Model Classification Report** \n"))
    print('\n',report)
    
for (model,report) in zip(model_list, report_list):
    display_report(model,report)
  

# 質問
**精度、検出率、F1 メトリクス **に基づいて、どの**モデル**が予測に最も適していると思いますか?

**ここで、特定のモデルに対して 2005 のデータを実行してください。**

# モデルのテスト

以下のコードを実行して**2005 データ**を確認します。

In [ ]:
df_2005.head(30)

特徴量選択ボックスで選択した特徴に基づく X_train および Y_train データを選択します。以下のリンクをクリックして、選択を確認できます。

<a href='#feature selection'>**特徴量選択ボックスに移動**</a>

In [ ]:
if feature_premiere_wide:
    X_train = df_2005[['rating', 'ratingCount', 'runtimeMinutes','premiere','wide']]
elif feature_premiere:
    X_train = df_2005[['rating', 'ratingCount', 'runtimeMinutes','premiere' ]]
elif feature_wide:
    X_train = df_2005[['rating', 'ratingCount',  'runtimeMinutes' ,'wide']]
else :
    X_train = df_2005[['rating', 'ratingCount',  'runtimeMinutes']]


Y_train = df_2005['nomination_winner']

if normalize_flag:
    X_train=(StandardScaler().fit_transform(X_train ))

データセットをトレーニングデータセットとテストデータセットに分割します。

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)

**選択したモデルを実行します**。前の手順で最適なモデルボックスからコードをコピーし、以下のボックスに貼り付けます。